# Error subspace SDP

In this ipynb file, we implement SDP for optimal recovery. We consider entanglement fidelity
\begin{align}
F_{\mathcal{N}_\gamma, \mathcal{R}}=\left\langle\Psi\left|\left(\mathcal{R} \circ \mathcal{N}_\gamma\right) \otimes \mathcal{I}(|\Psi\rangle\langle\Psi|)\right| \Psi\right\rangle.
\end{align}

SDP gives the optimal recodery
\begin{align}
F_{\mathcal{N}_\gamma, \mathcal{R}_{\mathrm{op}}}=\max _{\mathcal{R}}\left\langle\Psi\left|\left(\mathcal{R} \circ \mathcal{N}_\gamma\right) \otimes \mathcal{I}(|\Psi\rangle\langle\Psi|)\right| \Psi\right\rangle
\end{align}

Our implementation has following features:

1. the SDP has only one input: the QEC matrix.

2. the choi matrix is restricted on the error subspace. As a result, the choi matrix is $l*d*d$ dimensional, where l is the number of Kraus operators taken into consideration, $d=2$ for qubit code.

Also, we provide the results of transpose channel. 
\begin{align}
\mathcal{R}_T(\cdot) \equiv \sum_i P E_i^{\dagger} \mathcal{N}_\gamma(P)^{-1 / 2}(\cdot) \mathcal{N}_\gamma(P)^{-1 / 2} E_i P
\end{align}
Tranpose fidelity can serve as an approximation to the optimal fidelity, and transpose channel(in choi matrix form) serves as a good guess to optimized channel.

# Next step

1. use transpose channel as an initial guess to speed up solving SDP.
2. when iterate through gamma = np.linspace(0,0.1,11), use previous result as initial guess.

In [6]:
import numpy as np
from QECmat_GKP import QECmat_GKP
from process_QECmat import process_QECmat


# Construct QEC matrix
Here we use Gaussian envelope finite energy GKP code under pure loss as an example, we use the series expansion algorithm.

You can substitute it for other codes. The QEC matrix $M_{l,\mu,l',\mu'}$ should align as following(here we assume qubit code, and Kraus truncated by L):

$\left(\begin{array}{cccc}M_{00,00} & M_{00,01} & . . & M_{00,L1} \\ M_{01,00} & M_{01,01} & . . & M_{01,L1} \\ & & . . & . . \\ . . & . . & . \\ M_{L1,00} & M_{L 1,01} & . . & M_{L1,L1}\end{array}\right)$

In [7]:
####################
# calculate QEC matrix for GKP
####################
gamma = 0.05    # pure loss rate
Delta = 0.481   # 1/2 Delta^2 \approx photon number
l_cut = 20      # number of Kraus operator taken into consideration
m_sum_cutoff=20 # accuracy order of basis overlap
M_sum_cutoff=5  # accuracy order of QEC matrix

# calculate GKP QEC matrix, with orthonormal code basis
M_GKP = QECmat_GKP(Delta = Delta, gamma = gamma, l_cut = l_cut, m_sum_cutoff = m_sum_cutoff, M_sum_cutoff = M_sum_cutoff).orth_M()


# tranpose channel

According to our calculation, entanglement fidelity given by transpose channel is 
\begin{align}
F_{\mathcal{N}_\gamma, \mathcal{R}_T}=\frac{1}{4} \sum_{l, l^{\prime}}\left(\sum_\mu\left(\left(M^{\perp}\right)^{\frac{1}{2}}\right)_{l, \mu, l^{\prime}, \mu}\right)^2,
\end{align}
where $M^{\perp}$ is the error correction matrix in orthonormal basis(which we denote as $M$ in following sections). The choi matrix of $\mathcal{R}_T$ reads 
\begin{align}
(C_{\mathcal{R}_T})_{\nu,\nu';l,\mu,l',\mu'} = \delta_{l,l'}\delta_{\mu,\nu}\delta_{\mu',\nu'}
\end{align}
in our problem setting.

## code implementation
In our code, initialization method of 
```ruby
class process_QECmat
``` 
takes matrix $M$ as an input,  method 
```ruby
    def transpose_infid_M(self)
``` 
calculates the infidelity according to our formula, and method
```ruby
    def tranpose_choi(self)
``` 
calculates the choi matrix given by transpose recovery.

To test the consistency, we show that infidelity given by our expression and by transpose choi matrix are the same.

In [8]:
####################
# calculate transpose results
####################
GKP_QECprocess = process_QECmat(QECmat = M_GKP,dimL = l_cut,d = 2)  # input GKP QEC matrix

# calculate transpose fidelity result directly
trans_infid = GKP_QECprocess.transpose_infid_M()
print('transpose infid direct:',trans_infid)

# calculate transpose choi matrix, and test consistency
trans_choi = GKP_QECprocess.tranpose_choi()
prob0 = GKP_QECprocess.SDP_set_prob()  # setup SDP problem
prob0.variables()[0].save_value(trans_choi)# feed choi into the variable
print('transpose infid by choi:',1-prob0.objective.value)

transpose infid direct: 0.005356332498326677
transpose infid by choi: 0.005356332498326677


# SDP optimization

We can prove that SDP for optimal recovery channel can be written as
\begin{align}
&\max_{C_{\mathcal{R}}}\sum_{i,i',\mu,\mu',\nu,\nu'} (C_{\mathcal{R}})_{\nu,\nu';i,\mu,i',\mu'}K_{\nu,\nu';i,\mu,i',\mu'}, \\
&(C_{\mathcal{R}})_{\nu,\nu';i,\mu,i',\mu'} \ge 0, \\
&\sum_{\nu}(C_{\mathcal{R}})_{\nu,\nu;i,\mu,i',\mu'} = I_{i,\mu,i',\mu'},
\end{align}
where $I$ is the identity matrix,  $K_{\nu,\nu';i,\mu,i',\mu'} = \sum_{i_0,\mu_0,\nu_0}(M^{1/2})_{\nu,\nu_0;i,\mu,i_0,\mu_0}(M^{1/2})_{\mu_0,\nu';i_0,\nu_0,i',\mu'}$, and $(C_{\mathcal{R}})_{\nu,\nu';i,\mu,i',\mu'} \ge 0$ means that matrix $(C_{\mathcal{R}})_{\nu,\nu';i,\mu,i',\mu'} $ is positive semi definite with $(\nu,i,\mu)$ as row index, and $(\nu',i',\mu')$ as column index. (in principal, these $M^{1/2}$ can be cholesky decomposition, with which can reduce a factor of constant conplexity. But here we use square root for simplicity.)


## code implementation

Method 
```ruby
    def SDP_set_prob(self)
``` 
sets up the SDP problem and returns it. Using solve method in cvxpy, the problem will be solved.

In [9]:
####################
# SDP optimization
####################

prob1 = GKP_QECprocess.SDP_set_prob()  # setup SDP problem
prob1.solve(eps=1e-6,verbose = True )   # solve SDP
print('infid by SDP:',1-prob1.value,'iter:',prob1._solver_stats.num_iters)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Oct 11 11:39:09 AM: Your problem has 6400 variables, 1601 constraints, and 0 parameters.
(CVXPY) Oct 11 11:39:09 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 11 11:39:09 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 11 11:39:09 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 11 11:39:09 AM: Compiling problem (target solver=SCS).
(CVXPY) Oct 11 11:39:09 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> 

# more tests

In [10]:
for gamma in np.linspace(0,0.1,11):
    ####################
    # set parameters for GKP
    ####################
    print('--- gamma =',gamma)
    Delta = 0.481   # 1/2 Delta^2 \approx photon number
    l_cut = 20      # number of Kraus operator taken into consideration
    m_sum_cutoff=20 # accuracy order of basis overlap
    M_sum_cutoff=5  # accuracy order of QEC matrix
    
    ####################
    # calculate QEC matrix for GKP
    ####################
    M_GKP = QECmat_GKP(Delta = Delta, gamma = gamma, l_cut = l_cut, m_sum_cutoff = m_sum_cutoff, M_sum_cutoff = M_sum_cutoff).orth_M()
    
    ####################
    # calculate optimal/near optimal recovery
    ####################
    GKP_QECprocess = process_QECmat(QECmat = M_GKP,dimL = l_cut,d = 2)  # input GKP QEC matrix
    
    # calculate transpose fidelity result directly
    trans_infid = GKP_QECprocess.transpose_infid_M()
    print('transpose infid direct:',trans_infid)

    # calculate transpose choi matrix, and test consistency
    trans_choi = GKP_QECprocess.tranpose_choi()
    prob0 = GKP_QECprocess.SDP_set_prob()  # setup SDP problem
    prob0.variables()[0].save_value(trans_choi)# feed choi into the variable
    print('transpose infid by choi:',1-prob0.objective.value)
    

    # do SDP optimization
    prob1 = GKP_QECprocess.SDP_set_prob()  # setup SDP problem
    prob1.solve(eps=1e-6,verbose = False)   # solve SDP
    print('infid by SDP:',1-prob1.value,'iter:',prob1._solver_stats.num_iters)

--- gamma = 0.0
transpose infid direct: 0.0
transpose infid by choi: 1.1102230246251565e-16
infid by SDP: -7.451016417192591e-07 iter: 75
--- gamma = 0.01
transpose infid direct: 0.0007032768163498515
transpose infid by choi: 0.0007032768163499625
infid by SDP: 0.0006820488410448355 iter: 1150
--- gamma = 0.02
transpose infid direct: 0.0015788792758916204
transpose infid by choi: 0.0015788792758915093
infid by SDP: 0.0014937535378350963 iter: 875
--- gamma = 0.03
transpose infid direct: 0.002638626255126786
transpose infid by choi: 0.002638626255126786
infid by SDP: 0.0024444214089310856 iter: 1800
--- gamma = 0.04
transpose infid direct: 0.00389402609116718
transpose infid by choi: 0.00389402609116718
infid by SDP: 0.003547022365070873 iter: 1750
--- gamma = 0.05
transpose infid direct: 0.005356332498326677
transpose infid by choi: 0.005356332498326677
infid by SDP: 0.004812102153818998 iter: 1250
--- gamma = 0.06
transpose infid direct: 0.007036571652058288
transpose infid by choi: 0